# Negative News Neural Nets Project: Classifying Adverse Media Articles using Machine Learning Algorithms

In this notebook, conda environment with Python 3.86 is used. Some libraries, such as spacy and nltk may require installation if your machine does not have them. 

You can use the steps below to install spaCy. If something goes awry, feel free to use pip/do some stackoverflow search to complete the installation. The last two parts will be required later on in the notebook, they are not essential spaCy packages.

 - conda install -c conda-forge spacy
 
 - conda install -c conda-forge spacy-lookups-data
 
 - python -m spacy download en_core_web_sm
 
 - pip install spacy-langdetect
 
 - conda install -c conda-forge wordcloud
 
On the other hand, installing nltk packages will be easy, just look at the error to understand what needs to be downloaded using nltk.download(...). I have already provided the download code for punkt package and I don't think anything is required beside that.

## Dataset Preparation

Before doing any EDA, null value imputation, necessary dataset checks etc, we need to form the whole training dataset by combining the AM and NAM articles together. The latter one will include the random articles as well.

Let's begin with importing necessary/potentially useful stuff... Some of them below may not be used at all in the future, so the list below is tentative.

In [2]:
import warnings
warnings.simplefilter("ignore", UserWarning)

In [3]:
import pandas as pd
import numpy as np
import json
import spacy
import matplotlib.pyplot as plt
%matplotlib inline

# For regular expressions
import re

# For handling strings
import string

# For performing mathematical operations
import math

In [6]:
# Uncomment this if you're using linux
# !ls 

In [7]:
# Let's get an overview of what our folder contains..
!dir

 Volume in drive C is Win 10
 Volume Serial Number is CA9A-F06E

 Directory of C:\Users\canberk\Desktop\ut-ml-adverse-media-main

12/07/2020  05:51 PM    <DIR>          .
12/07/2020  05:51 PM    <DIR>          ..
11/23/2020  06:08 PM           110,455 .ipynb
11/23/2020  06:09 PM    <DIR>          .ipynb_checkpoints
11/21/2020  04:43 PM         3,752,073 adverse_media_training.csv.zip
11/21/2020  07:29 PM         3,097,536 cleaned&tokenized_text.csv
12/07/2020  05:51 PM           134,333 Data Preprocessing&Baselines.ipynb
11/21/2020  04:43 PM         3,630,748 EDA - Kristjan's Original.ipynb
11/23/2020  06:04 PM         3,740,422 EDA.ipynb
10/24/2015  07:35 PM     5,646,236,541 glove.840B.300d.txt
12/07/2020  04:36 PM     2,176,768,927 glove.840B.300d.zip
11/21/2020  04:43 PM             1,073 LICENSE
11/21/2020  04:43 PM         3,764,231 non_adverse_media_training.csv.zip
11/21/2020  04:43 PM                21 README.md
              11 File(s)  7,841,236,360 bytes
               3 Di

We can see that the data required are in zipped format. Let's read them with pandas.

In [6]:
am = pd.read_csv('adverse_media_training.csv.zip')
nam = pd.read_csv('non_adverse_media_training.csv.zip')

Let's check the labels in both datasets. We may(/will :)) encounter some typos among them.

In [7]:
print(am.label.unique())
print()
print(am.label.value_counts())

[nan 'am' 'delete' 'delete        ' 'random' 'nam' 'doubt' 'neither'
 'am, doubt' 'am ']

am                391
delete             32
nam                18
random             12
doubt               5
neither             2
delete              2
am, doubt           1
am                  1
Name: label, dtype: int64


In [8]:
print(nam.label.unique())
print()
print(nam.label.value_counts())

['nam' nan 'am' 'neither' 'random' 'doubt' 'delete']

nam        285
am          19
doubt       13
delete       9
neither      7
random       3
Name: label, dtype: int64


Both datasets are not pure in their essence. We need to transfer some rows between them and drop the unnecessary rows having labels such as 'delete', 'neither' etc.

In [9]:
# Creating the AM dataset for train
am_confirmed = am.loc[(am.label == 'am') | (am.label == 'am ')]
am_confirmed = pd.concat([am_confirmed, nam.loc[nam.label == 'am']])
am_confirmed.shape

(411, 11)

In [10]:
# Creating NAM dataset for train
nam_confirmed = nam.loc[(nam.label == 'nam') | (nam.label == 'random')]
nam_confirmed = pd.concat([nam_confirmed, am.loc[(am.label == 'nam') | (am.label == 'random')]])
nam_confirmed.shape

(318, 11)

In [11]:
# Let us also append the necessary labels. Actually, we can also modify the label column in both datasets directly.
am_confirmed['is_adverse_media'] = 1
nam_confirmed['is_adverse_media'] = 0

In [12]:
# Creating the train dataset
train = pd.concat([am_confirmed, nam_confirmed])
print(train.shape)
print()
print(train['is_adverse_media'].value_counts())

(729, 12)

1    411
0    318
Name: is_adverse_media, dtype: int64


In [13]:
# Ratio of AM to NAM class
print('Ratio of AM to NAM articles:', round(411/318, 2))

Ratio of AM to NAM articles: 1.29


Our dataset may turn out to be small for now, but thankfully it is not imbalanced very much. 

**After adding Oskar's json data, dataset imbalance will be a problem.**

Anyway, let's take a quick look into what type of columns the train set has, and get some sumamry statistics on the labels.

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 729 entries, 8 to 773
Data columns (total 12 columns):
source              729 non-null object
entity_name         630 non-null object
entity_type         631 non-null object
url                 729 non-null object
article             729 non-null object
full_response       729 non-null object
label               729 non-null object
explanation         633 non-null object
assessor            727 non-null object
comment             23 non-null object
title               729 non-null object
is_adverse_media    729 non-null int64
dtypes: int64(1), object(11)
memory usage: 74.0+ KB


In [15]:
train.is_adverse_media.describe()

count    729.000000
mean       0.563786
std        0.496255
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: is_adverse_media, dtype: float64

Since our basic task is to classify text articles, we will need the *article* and *is_adverse_media* columns for the sentiment analysis task. 

Later on, if we decide to add an entity recognition task or turn this into a multilabel classification problem, we will need some other columns like *entity_name* as well.

## Data Preprocessing

Until now, we have created the training dataset in its crude form. In this part we will filter the training data and check the articles column for null values or non-english text.

In [16]:
train.head()

,source,entity_name,entity_type,url,article,full_response,label,explanation,assessor,comment,title,is_adverse_media
8,Canberk,Sam Waksal,Person,http://content.time.com/time/specials/packages...,"Bernie Madoff, who is scheduled to be sentence...","[{'error': 'Proxy error: msgtimeout', 'query':...",am,fraud,Carel,NaN,Top 10 Crooked CEOs,1
10,Dan,Mark Denning,Person,https://www.bbc.com/news/business-50089887,Published\n\nOne of the world's leading fund m...,[{'query': {'id': '1605053818341-a38bb1c20fc7b...,am,broke investment rules,Wanting,NaN,Top fund manager forced to resign after BBC in...,1
11,Darya,Russell Wasendorf Sr,individual,https://www.bbc.com/news/business-19631611,Published\n\nThe founder of US futures broker ...,[{'query': {'id': '1605055958079-99347130d4bde...,am,pleads guilty to fraud,Sebastien,NaN,Peregrine Financial Group boss admits $100m fraud,1
12,Karl,NaN,NaN,https://apnews.com/article/9acaa6485cbe480d843...,WASHINGTON (AP) — An American security contrac...,[{'query': {'id': '1605050186817-549afd2bcf473...,am,"Corruption, multiple people",Karl,NaN,American accuses Congo officials of unlawful a...,1
17,Canberk,Charlie Shrem,Person,https://www.theguardian.com/technology/2014/ja...,"A senior figure in the Bitcoin Foundation, whi...",[{'query': {'id': '1605374115413-cdb55925327dd...,am,arrested for money launering,Sebastien,NaN,Bitcoin Foundation vice chair arrested for mon...,1


Let's drop the unnecessary columns from the training dataset. 

(**Question for Kristjan:** Do the columns 'url, full_response and title' necessary for any other extra analysis in the future?)

In [17]:
# Keep only needed columns
train = train.loc[:, ['entity_name', 'entity_type', 'url', 'article', 'full_response', 'explanation', 'title', 'is_adverse_media']]

train.describe(include='all')

,entity_name,entity_type,url,article,full_response,explanation,title,is_adverse_media
count,630,631,729,729,729,633,729,729.000000
unique,483,16,729,729,729,370,723,NaN
top,John McAfee,individual,https://www.civilserviceindia.com/subject/Gene...,Banks have given a cautious welcome to US move...,[{'query': {'id': '1605044480200-60ffcd0dc7f34...,corruption,Ministry of Justice calls upon law firms to ta...,NaN
freq,8,185,1,1,1,34,2,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.563786
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.496255
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000


Now we can narrow our focus a little bit more... Let's check if there are any nulls in article & is_adverse_media columns.

In [18]:
sum(train.article.isna()), sum(train.is_adverse_media.isna())

(0, 0)

We gotta do one last check before tokenizing the articles, we need to check if there are any non-english text managed to slip in during the data collection process. spaCy can do this with its langdetect module, hope you succeeded in installing it.

In [19]:
#!pip install spacy-langdetect

In [20]:
# Make sure we only have English articles
from spacy_langdetect import LanguageDetector

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

# Let's create an example doc object first to test spaCy's LanguageDetector.
text = 'This is an english text. Ja see on eestikeelne lause. بوغيث	'
doc = nlp(text)
# document level language detection. Think of it like average language of the document!
print(doc._.language)
# sentence level language detection
for sent in doc.sents:
    print(sent, sent._.language)

{'language': 'et', 'score': 0.9999948922362369}
This is an english text. {'language': 'en', 'score': 0.9999966325242425}
Ja see on eestikeelne lause. {'language': 'et', 'score': 0.9999937782572812}
بوغيث	 {'language': 'ar', 'score': 0.9999945969262914}


The language detector acts like a weirdo while trying to understand the average language of the document. Classifying the whole text as 85 percent Estonian is a bit too much in my opinion. Let's test it on an actual article in the train dataset.

In [21]:
example = train.article[8]
example # Clearly this one's in English.

"Bernie Madoff, who is scheduled to be sentenced June 29 for perpetrating history's biggest Ponzi scheme, is just be the latest in a long line of industry titans turned crooks\n\nCRIMINAL EXECUTIVE OFFICER\nSam Waksal\n\nCEO:\xa0ImClone\nConvicted: October 15, 2002 of securities fraud, bank fraud, obstruction of justice, and perjury\nKnown for his networking skills as much as for his scientific expertise, immunologist Sam Waksal founded ImClone in 1984. The New York-based biotech firm remained relatively unknown until 1999, when it announced the creation of Erbitux — a cancer-fighting drug so promising it convinced pharmaceutical giant Bristol-Myers to purchase $1 billion of ImClone stock in one of the largest biotechnology partnerships in U.S. history. But when the Food and Drug Administration rejected the drug, Waksal alerted several relatives and friends to dump their stock as soon as possible — before the FDA's decision had been made public. Waksal's father and daughter sold $9.2 m

In [22]:
doc = nlp(example)
print(doc._.language)

{'language': 'en', 'score': 0.9999962549118911}


With long texts like the articles we collected, spaCy does a good job.

Now, let's check the whole dataset to see if any non-english article exists.

In [23]:
train['article'].apply(lambda article: nlp(article)._.language['language']).unique()

array(['en'], dtype=object)

All of our articles are in English. We can now move on to creating tokens.

Before applying any vectorizer, we need to create tokens from our articles by cleaning them from punctuation, empty spaces etc. The helper function below will use some regex commands to handle all those, besides transforming all the letters to lowercase.

In [29]:
# The regex below can be modified later on.
def lemmatize(article):
    article = re.sub(r'http\S+', '', article)
    article = re.sub(r"#(\w+)", '', article)
    article = re.sub(r"@(\w+)", '', article)
    article = re.sub(r'[^\w\s]', '', article)
    article = re.sub(r'\w*\d\w*','', article)
    article = re.sub(' +',' ', article)
    article = article.strip().lower()
    
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    doc = nlp(article)
    lemmatized_article = " ".join([token.lemma_ for token in doc if (token.is_stop==False)]) 
    
    return lemmatized_article

Again, let's use the lemmatize function on an example to see what it does.

In [34]:
example = train.article[8]
lemmatized = lemmatize(example)

print('Before Lemmatization:')
print()
print(example)
print()

print('After Lemmatization:')
print()
print(lemmatized)
print()

Before Lemmatization:

Bernie Madoff, who is scheduled to be sentenced June 29 for perpetrating history's biggest Ponzi scheme, is just be the latest in a long line of industry titans turned crooks

CRIMINAL EXECUTIVE OFFICER
Sam Waksal

CEO: ImClone
Convicted: October 15, 2002 of securities fraud, bank fraud, obstruction of justice, and perjury
Known for his networking skills as much as for his scientific expertise, immunologist Sam Waksal founded ImClone in 1984. The New York-based biotech firm remained relatively unknown until 1999, when it announced the creation of Erbitux — a cancer-fighting drug so promising it convinced pharmaceutical giant Bristol-Myers to purchase $1 billion of ImClone stock in one of the largest biotechnology partnerships in U.S. history. But when the Food and Drug Administration rejected the drug, Waksal alerted several relatives and friends to dump their stock as soon as possible — before the FDA's decision had been made public. Waksal's father and daughter

Let's copy our train data and apply lemmatization on the articles belonging to the copy.

In [35]:
data = train[['article', 'is_adverse_media']].copy()
data = data.reset_index()
data = data.drop(['index'], axis=1)
print('Shape of our DataFrame:', data.shape)

data.head()

Shape of our DataFrame: (729, 2)


,article,is_adverse_media
0,"Bernie Madoff, who is scheduled to be sentence...",1
1,Published\n\nOne of the world's leading fund m...,1
2,Published\n\nThe founder of US futures broker ...,1
3,WASHINGTON (AP) — An American security contrac...,1
4,"A senior figure in the Bitcoin Foundation, whi...",1


In [36]:
data['lemmatized_articles'] = data['article'].map(lemmatize)
data.head()

,article,is_adverse_media,lemmatized_articles
0,"Bernie Madoff, who is scheduled to be sentence...",1,bernie madoff schedule sentence june perpetrat...
1,Published\n\nOne of the world's leading fund m...,1,publish \n\n world lead fund manager force res...
2,Published\n\nThe founder of US futures broker ...,1,publish \n\n founder future broker peregrine f...
3,WASHINGTON (AP) — An American security contrac...,1,washington ap american security contractor acc...
4,"A senior figure in the Bitcoin Foundation, whi...",1,senior figure bitcoin foundation lobby behalf ...


In [37]:
data = data[['is_adverse_media', 'lemmatized_articles']] # These are the only columns that we need for modeling
data = data.sample(frac = 1) # Let us not forget to shuffle the rows before train_test_split

print('Shape of our DataFrame:', data.shape)

data.head()

Shape of our DataFrame: (729, 2)


,is_adverse_media,lemmatized_articles
481,0,fraud manager access seon new intelligence too...
214,1,washington ap treasury name mob boss italys ca...
110,1,\n\n min read \n\n miami reuters court senten...
306,1,san diego rabbi yisroel goldstein director cha...
721,0,nauru bank field mineralrich smooth brown rock...


Let's save this cleaned dataframe as a csv file for later usage.

In [1]:
# Don't run this line again, the csv file is already created. This is just for explanatory purposes.
# data[['is_adverse_media', 'lemmatized_articles']].to_csv('./cleaned_lemmatized_text.csv')